This notebook contains some demos.

**Note to self:** `jupyter nbconvert --to rst text/demo_notebook.ipynb` (requires  `pandoc`).

# 1. Error handling

To report an error (which in Python, is called an `Exception`), just `raise` it:

In [ ]:
import math

def _sqrt(x: float, tol: float = 1e-5) -> float:
    """Computes the square root of a number `x` up to a given convergence `tol`, using the Newton algorithm.
    
    :math:`x_{n+1} = x_n - \\frac{f(x_n)}{f'(x_n)}`, where :math:`f(x)=x^2-a`.
    
    Arguments:
        x: a floating point number
    Returns:
        the square root of `x`
    Raises:
        ValueError: if `x` is not a number
    """
    if x < 0:
        raise ValueError("x must be positive")
    a = 1
    while abs(a**2 - x) > tol:
        a = .5 * (a + x / a)
    
    return a

assert abs(_sqrt(15) - math.sqrt(15)) < 1e-5

# also try this: assert abs(_sqrt(3) - math.sqrt(1)) < 1e-5

It is always nice to give a bit of context, so the first (and generally only) argument of an `Exception` is an error message.

So, what happen when you use a negative number?

In [ ]:
print(_sqrt(-2))

The exception was raised, interupting the process. In fact, the square root is never computed and nothing is printed!

You generally get a *stacktrace*, which is relatively useful in order to debug.

But it is sometimes useful to *catch* the error and act accordingly. To do so, put the call in a `try`/`except` bloc:

In [ ]:
try:
    _sqrt(-2)
except ValueError as e:
    print('Caught this:', e)

Notice that you can *catch* the error. This is useful to nicely report the error to the user. But you can do other things:

In [ ]:
try:
    _sqrt(-3)
except ValueError as e:
    print('This square root is imaginary, and its value is', _sqrt(3), 'i')

Note that you *catch* only the error indicated after `except`. For example,

In [ ]:
try:
    _sqrt('test')
except ValueError as e:
    print('caughth this:', e)

The `TypeError` was not *caught* and continue its way up to the main process. You can catch different type of errors by using the following construction:

In [ ]:
try:
    _sqrt('test')
except (ValueError, TypeError) as e:
    print('caught this:', e)

As you can see, the python objects also uses exceptions, which means that there is a bunch of already defined exceptions (postfixed by `Error`) which are available (see a list at https://docs.python.org/3/library/exceptions.html#concrete-exceptions). You can choose any of them for your own functions.

## Custom exceptions

You can define your own Exceptions, by deriving from `Exception`:

In [ ]:
class CustomException(Exception):
    pass

class AnotherCustomException(CustomException):
    def __init__(self, value: int, problem: str):
        super().__init__('Error with {}: {}'.format(value, problem))

raise AnotherCustomException(10, 'custom error')

Note that the inheritance applies to `except`, so:

In [ ]:
try:
    raise AnotherCustomException(10, 'custom error')
except CustomException as e:  # this catches AnotherCustomException as well!
    print('caught this:', e)

Finally, note that an error that is not *caught* crawls up in the stack trace. For example,

In [ ]:
def a():
    _sqrt(-2)

def b():
    try:
        a()
    except AnotherCustomException as e:
        print('error reported in b()', e)

def c():
    try:
        b()
    except ValueError as e:
        print('Error reported in a():', e)

c()  # calls `b()` which itself calls `a()`

The error is *raised* in `a()` but only *caught* in `c()` since, despite a `try`/`except` block in `b()`, the Exception is not *caught* there.

## Python principle: "*easier to ask for forgiveness than permission*"

Python as a few maxims like this. This one is well explained in [this *stackoverflow* answer](https://stackoverflow.com/a/11360880):

In [ ]:
my_dict = {}

# EAFP:
try:
    x = my_dict['key']
except KeyError:
    pass

# LBYL ("Look before you leap"):
if 'key' in my_dict:
    x = my_dict['key']
else:
    pass


The "LBYL" version has to search the key inside the dictionary **twice**... and might be considered slightly less readable by some people ;)

# 2. Classes and objects

To define an ADT, use the `class` keyword. 

Remember, there are 3 steps:

1. specify your ADT,
2. specify its methods, and
3. choose an irep that fits your specification.

Here, my goal is to have an object that represent a tensor, $T^{(n)}$ of rank $n$, and to compute the response, $\vec r$, to a perturbation $\vec p$, such that:

$$r_i = \sum_{jk\ldots} T_{ij\ldots}\,p_j\,p_k\ldots$$

(think unit sphere representation)

In [ ]:
import numpy as np

class ResponseTensor:
    """
    Represent a response tensor of rank `n`.
    Allow to compute the response to a perturbation.
    
    Invariant:
        This ADT is immutable.
    """
    
    def __init__(self, rank: int, components: np.ndarray):
        """
        Create a new `ResponseTensor` instance of rank `n`.
        
        Args:
             rank: the rank of the tensor
             components: the components of the tensor, must have `len(components.shape) == rank`
        """
        
        assert len(components.shape) == rank
        
        self.rank = rank
        self.components = components
    
    def response(self, perturbation: np.ndarray) -> np.ndarray:
        """
        Compute the response to a perturbation:
        
        :math:`r_i = \sum_{jk\ldots} T_{ijk\ldots}\\,p_j\\,p_k\ldots`.
        
        Args:
            perturbation: the perturbation, must have ``self.components.shape[-1] == perturbation.shape[-1]``.
        Returns:
            $r$, the response to the perturbation.
        """
        
        assert self.components.shape[-1] == perturbation.shape[-1]
        
        r = self.components.copy()
        for i in range(self.rank - 1):
            r = r @ perturbation
        
        return r
    
    def __mul__(self, n: int|float):
        """
        Create a new `ResponseTensor` instance whose components are multiplied by `n`.
        
        Args:
            n: the number to multiply
        Returns:
            A new `ResponseTensor` instance whose components are multiplied by `n`.
        """
        assert type(n) in [int, float]
        
        return ResponseTensor(self.rank, self.components * n)
    
    def __repr__(self):
        return '<ResponseTensor(rank={})>'.format(self.rank)
    
    def __str__(self):
        return 'Response tensor of rank={}:\n{}'.format(self.rank, self.components)

Notice that:

+ The *irep* (member variables) is defined through the **constructor**, `__init__(self, ...)`. Here, nothing complicate, we simply store the components and the rank of the tensor. 
+ The constructor defines how you will create instances of this ADT (**objects**). Here, you have to provide two arguments.
+ The first argument of all methods, `self`, is an argument to refer to the *instance* on which the method acts.
+ Along the constructor, there are others *magic* methods that are defined.
+ The invariant tells us that the instances are **immutable** (the tensor cannot change). This means that this class needs *producers* method (which return a new instance) rather than *modifiers* methods (which modifies in-place). See `__mul__()`.

To use the ADT, we create an instance, also called an **object**, of it:

In [ ]:
tensor1 = ResponseTensor(2, np.eye(3))
tensor2 = ResponseTensor(2, 2 * np.eye(3) - .25 * np.ones((3, 3)))

# compute a response to a perturbation, call `response()`:
perturbation = np.array([1., 2., .0])
perturbation /= np.linalg.norm(perturbation)
response = tensor2.response(perturbation)
print('response vector:', response)

# request its string representation, call `__str__()`
print('string representation:', str(tensor2))

# request a representation, call `__repr__()`
print('repr of list:', [tensor1, tensor2])

# use `__mul__()`
print('after multiplication:', tensor1 * 3)


**Note:** It is totally allowed to directly access the components using `tensor1.components`. However, this is **not recommended**, since `components` is part of the *irep*, and thus "private". 

The developer is responsible for not breaking the API (here, the methods of the class) too much, but it can change the *irep* at any time (it is **not** part of the specification), so if your code relies on directly accessing or modifying *irep*, it might be broken at any update (and, per *programming by contract*, it is not the fault of the developer but **yours**).

In this specific case, if you modify `tensor1.components`, you actually break the *invariant*, which means that per *programming by contract*, it is not guaranteed that the implementation will work anymore. And, indeed:

In [ ]:
tensor1.components = np.zeros((3, 3, 3))  # accessing the irep is not forbidden per se
print(tensor1)  # ... But you expose yourself to incoherence

In some languages (C++, Java, etc.), the separation between "public" and "private" is enforced by the compiler (... But the irep is just a memory zone, so one can still access it if it wants, it is not a security measure). In Python, this is not the case, but the good practice is to mark private variable by starting with an underscore, *e.g.* `_components` (and thus some IDE give a warning when you access an underscore-prefixed variable). This applies to methods, but also package-wide private functions and variables (think *matplotlib*).

## Inheritance

To specialize your ADT, you can use inheritance:

In [ ]:
class PolarizabilityTensor(ResponseTensor):
    """
    A polarizability tensor (second-order response to an electric field).
    
    Invariant:
        The rank of this tensor is 2, and it has 3x3 components.
    """
    
    def __init__(self, components: np.ndarray):
        """Create a new instance of `PolarizabilityTensor`.
        
        Args:
            components: the components of the tensor, must have `components.shape == (3, 3)`.
        """
        
        assert components.shape == (3, 3)
        
        super().__init__(2, components)
        
    def __str__(self):
        return 'Polarizability tensor:\n{}'.format(self.components)
    
    def invariant_iso(self):
        """
        Returns:
             the isotropic value of the polarizability tensor.
        """
        
        # thanks to the invariant, I do not have to check that this is indeed a polarizability tensor
        return np.trace(self.components) / 3

Notice that:

+ The inherited class (marked with `class Child(Parent)`) inherits **all** methods and members.
+ One can redefine some methods (here, `__init__()` and `__str__()`). The one that are not redefined (here, `response()`) act exactly as the one in the parent class. One can access the "parent" version by using `super().method()` (see constructor).
+ New methods can be defined as well. They are only available for the child.
+ It is ok to access the *irep* of the parent in the child. The child can have its own *irep*, thought.
+ Remember: the child should never *weaken* the parent's invariant (here, the objects should never become mutable), only stregthen it (here, we imposed extra conditions), so that they can be interchanged. In practice, **it is not always possible**. 

Example of usage:

In [ ]:
tensor3 = PolarizabilityTensor(np.eye(3) - .5 * np.ones((3, 3)))
print(tensor3)

print('isotropic value:', tensor3.invariant_iso())

response = tensor3.response(perturbation)
print('response vector:', response)

Again, the idea is to be able to use `PolarizabilityTensor` at each place where `ResponseTensor` can be used. However, if you need (for some reason) to differenciate between the two, you can use the *reflexivity* capabilities of Python:

In [ ]:
print(isinstance(tensor3, ResponseTensor))  # true, because of inheritance
print(isinstance(tensor3, PolarizabilityTensor))  # true
print(isinstance(tensor2, ResponseTensor))  # true
print(isinstance(tensor2, PolarizabilityTensor))  # false

def f1(tensor: ResponseTensor):
    """
    This function accept any kind of response tensor. 
    No need to specify each type in the declaration, as a parent or child should be indistinguishable in this function.
    This means that the developer should only call methods of the parent.
    """
    pass

def f2(tensor: PolarizabilityTensor):
    """
    This function accepts only `PolarizabilityTensor`.
    Here, one can use the full set of methods available for this specific type.
    """
    pass

## Using OOP

In practice, objects are treated as "convenient big black boxes with methods" and OOP is used at most occasion, even if the object is not very "physical" (*e.g.*, a method such as HF or DFT can be an object, so that they share similar methods).

It is particularly useful if you want to define a set of common properties for your objects. This is called an interface:

In [ ]:
class IMolecule:
    """
    Interface for a molecule, a group of atoms, each with their coordinates.
    """
    
    def move(self, displacement: np.ndarray):
        """
        Move the whole molecule by ``displacement``
        
        Args:
            displacement: the displacement vector, must have `len(displacement) == 3`.
        """
        raise NotImplementedError()
    
class Atom:
    """An atom, which as a atomic number and a coordinate"""
    
    def __init__(self, Z: int, position: np.ndarray):
        """
        Create a new `Atom` instance.
        
        Args:
            Z: the atomic number, must be `> 0`
            position: the coordinates of the atom, must have `len(position) == 3`.
        """
        self.Z = Z
        self.position = position

class MoleculeWithAtomsImpl(IMolecule):
    """
    Concrete implementation of `IMolecule`, as a list of atoms.
    It is easy to add new atom to that one, but other modifications might be slower.
    """
    def __init__(self, atoms: list[Atom]):
        self.atoms = atoms
    
    def move(self, displacement: np.ndarray):
        for atom in self.atoms:
            atom.position += displacement


class MoleculeCompactImpl(IMolecule):
    """
    Concrete implementation of `IMolecule`, as list of Z + coordinates.
    It is easy to act on the coordinates of that one, but adding/removing atoms is difficult.
    """
    def __init__(self, Z: list[int], positions: np.ndarray):
        """
        Create a new `MoleculeCompactImpl` instance.
        
        Args:
            Z: list of atomic numbers
            positions: the coordinates of the molecule, must have `positions.shape == (len(Z), 3)`.
        """
        
        self.Z = Z
        self.positions = positions
    
    def move(self, displacement: np.ndarray):
        self.positions += displacement

Here, no matter the implementation, all childs should define `move()` with the same signature.